In [12]:
import pandas as pd
import re


In [13]:
# Step 1: Load data from CSV and handle encoding issues
file_path = '/content/drive/MyDrive/Portfolio Project/RF Jumper Grouping (Example).csv'

In [14]:
# Handle potential encoding issues
try:
    data = pd.read_csv(file_path, encoding='utf-8')
except UnicodeDecodeError:
    try:
        data = pd.read_csv(file_path, encoding='latin-1')
    except UnicodeDecodeError:
        data = pd.read_csv(file_path, encoding='ISO-8859-1')

In [15]:
# Ensure only relevant columns are selected
df = pd.DataFrame(data, columns=['Item Code', 'Autocount Name', 'Stock Count'])

In [16]:
print (df.head)

<bound method NDFrame.head of     Item Code                                     Autocount Name  Stock Count
0    40500044  7/16 FEMALE 1/2 TO 4.3-10 MALE 1/2" SUPERFLEX ...          NaN
1    40500078  7/16 MALE TO 7/16 MALE 1/2" SUPERFLEX JUMPER 4...        196.0
2    40500092  7/16 MALE TO 7/16 MALE 1/2" SUPERFLEX JUMPER 5...        102.0
3    40500079  7/16 MALE TO 7/16 MALE 1/2" SUPERFLEX JUMPER 8...        146.0
4    40500080  7/16 MALE TO 7/16 MALE 1/2" SUPERFLEX JUMPER 1...        187.0
5    40500081    N MALE TO 7/16 MALE 1/2" SUPERFLEX JUMPER 4 MTR         34.0
6    40500095    N MALE TO 7/16 MALE 1/2" SUPERFLEX JUMPER 5 MTR         30.0
7    40500162    N MALE TO 7/16 MALE 1/2" SUPERFLEX JUMPER 6 MTR         83.0
8    40500082    N MALE TO 7/16 MALE 1/2" SUPERFLEX JUMPER 8 MTR         69.0
9    40500163   N MALE TO 7/16 MALE 1/2" SUPERFLEX JUMPER 10 MTR         25.0
10   40500083    N MALE TO7/16 MALE 1/2" SUPERFLEX JUMPER 12 MTR         50.0
11   40500086  RF JUMPER ASSEMBLY 

In [17]:
# Function to extract base and length (e.g., SUPERFLEX JUMPER and 3MTR)
def extract_base_and_length(item):
    match = re.search(r"(.+?)\s(\d+(\.\d+)?\s*(MTR|METER))", item, re.I)
    if match:
        base = match.group(1).strip()
        length = match.group(2).strip()
        return base, length
    return item, None


In [18]:
# Function to identify connector type (without including any length terms)
def extract_connector_type(item):
    # Search for descriptions before SUPERFLEX/SUPERFLEXIBLE JUMPER
    before_base = re.search(r"(.*?)\s(SUPERFLEX|SUPERFLEXIBLE)\sJUMPER", item, re.I)
    after_base = re.search(r"(SUPERFLEX|SUPERFLEXIBLE)\sJUMPER\s(.*)", item, re.I)

    connector_types = []

    if before_base:
        connector_types.append(before_base.group(1).strip())

    if after_base:
        # Only include parts without length (no MTR/METER terms)
        after_connector = re.sub(r'\d+\s*(MTR|METER)', '', after_base.group(2)).strip()
        connector_types.append(after_connector)

    return " | ".join([x for x in connector_types if x])


In [19]:
# Apply extraction logic row-wise on the DataFrame
df[['Base', 'Length']] = df['Autocount Name'].apply(lambda x: pd.Series(extract_base_and_length(x)))
df['Connector Type'] = df['Autocount Name'].apply(lambda x: extract_connector_type(x))

In [20]:
print (df.head(10))

  Item Code                                     Autocount Name  Stock Count  \
0  40500044  7/16 FEMALE 1/2 TO 4.3-10 MALE 1/2" SUPERFLEX ...          NaN   
1  40500078  7/16 MALE TO 7/16 MALE 1/2" SUPERFLEX JUMPER 4...        196.0   
2  40500092  7/16 MALE TO 7/16 MALE 1/2" SUPERFLEX JUMPER 5...        102.0   
3  40500079  7/16 MALE TO 7/16 MALE 1/2" SUPERFLEX JUMPER 8...        146.0   
4  40500080  7/16 MALE TO 7/16 MALE 1/2" SUPERFLEX JUMPER 1...        187.0   
5  40500081    N MALE TO 7/16 MALE 1/2" SUPERFLEX JUMPER 4 MTR         34.0   
6  40500095    N MALE TO 7/16 MALE 1/2" SUPERFLEX JUMPER 5 MTR         30.0   
7  40500162    N MALE TO 7/16 MALE 1/2" SUPERFLEX JUMPER 6 MTR         83.0   
8  40500082    N MALE TO 7/16 MALE 1/2" SUPERFLEX JUMPER 8 MTR         69.0   
9  40500163   N MALE TO 7/16 MALE 1/2" SUPERFLEX JUMPER 10 MTR         25.0   

                                                Base  Length  \
0  7/16 FEMALE 1/2 TO 4.3-10 MALE 1/2" SUPERFLEX ...   2 MTR   
1 

In [21]:
# Print the DataFrame to verify everything
print("\n✅ Final Dataset with Extracted Columns")
print(df[['Item Code', 'Autocount Name', 'Stock Count', 'Base', 'Connector Type', 'Length']])


In [22]:
# Save the resulting DataFrame to a CSV file
output_file = '/content/grouped_products_with_connector.csv'
try:
    df.to_csv(output_file, index=False)
    print(f"\nGrouped data saved successfully to {output_file}")
except Exception as e:
    print(f"Failed to save grouped data to {output_file}: {e}")


Grouped data saved successfully to /content/grouped_products_with_connector.csv
